In [7]:
import zarr
import numpy as np
#Loading Required Libraries 
import anndata
import scipy as sp
#import geopandas
import dask 
from dask.distributed import Client
import xarray as xr
import pyseq
import skimage
from pyseq import image_analysis as ia
import torch
import pickle

In [8]:
labels = skimage.io.imread('/gpfs/commons/groups/nygcfaculty/PySeq/20210428_mouse_genotype_2/segmented_sections/m387ntga2_labels.tiff')
im = ia.get_HiSeqImages(image_path = 'zarrs/m387ntga2.zarr')

ImageAnalysis::Opened m387ntga2 


/gpfs/commons/home/jsingh/PySeq2500/pyseq/image_analysis.py:955: RuntimeWarning: Failed to open Zarr store with consolidated metadata, falling back to try reading non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  im = xr.open_zarr(fn).to_array()


In [9]:
#format = one_z_plane_obj_step_channel_cycle
array_object_list = []
name_list = []
for i in im.im['channel'].values:
    for j in im.im['cycle'].values:
        #for k #in im.im['obj_step'].values[1]:
            k = 8028
            nme = "one_z_plane_"+str(i)+"_"+str(j)+"_"+str(k)
            name_list.append(nme)
            nme = im.im.sel(obj_step = k, cycle=j, channel = i)
            array_object_list.append(nme)

In [10]:
#plane_mean_dict = {}
#ar = array_object_list[0].values
#tr = torch.from_numpy(ar)
#lab = torch.from_numpy(labels)

In [5]:
mx = labels.max()

In [6]:
def get_mean_intensity(pl):
    result_ar = np.zeros(mx)
    tr = torch.from_numpy(pl)
    for r in range(15):
        result_ar[r] = (tr[lab == r+1]).float().mean()
    return result_ar

In [15]:
lab = torch.from_numpy(labels)
plane_mean_dict = {}
for i,nm in zip(range(len(array_object_list)),name_list):
    print(i,nm)
    pl = array_object_list[i].values
    mean_int = get_mean_intensity(pl)
    plane_mean_dict.update({nm:mean_int})
    with open('saved_dictionary.pkl', 'wb') as f:
        pickle.dump(plane_mean_dict, f)
        


    
    
    
    

0 one_z_plane_558_1_8028
1 one_z_plane_558_2_8028
2 one_z_plane_558_3_8028
3 one_z_plane_558_4_8028
4 one_z_plane_610_1_8028


KeyboardInterrupt: 

In [2]:
import pickle

In [14]:
with open('mean_intensities.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

In [15]:
 loaded_dict.keys()

dict_keys(['one_z_plane_558_1_8028', 'one_z_plane_558_2_8028', 'one_z_plane_558_3_8028', 'one_z_plane_558_4_8028', 'one_z_plane_610_1_8028', 'one_z_plane_610_2_8028', 'one_z_plane_610_3_8028', 'one_z_plane_610_4_8028', 'one_z_plane_687_1_8028', 'one_z_plane_687_2_8028', 'one_z_plane_687_3_8028', 'one_z_plane_687_4_8028', 'one_z_plane_740_1_8028', 'one_z_plane_740_2_8028', 'one_z_plane_740_3_8028', 'one_z_plane_740_4_8028'])

In [16]:
name_list 

['one_z_plane_558_1_8028',
 'one_z_plane_558_2_8028',
 'one_z_plane_558_3_8028',
 'one_z_plane_558_4_8028',
 'one_z_plane_610_1_8028',
 'one_z_plane_610_2_8028',
 'one_z_plane_610_3_8028',
 'one_z_plane_610_4_8028',
 'one_z_plane_687_1_8028',
 'one_z_plane_687_2_8028',
 'one_z_plane_687_3_8028',
 'one_z_plane_687_4_8028',
 'one_z_plane_740_1_8028',
 'one_z_plane_740_2_8028',
 'one_z_plane_740_3_8028',
 'one_z_plane_740_4_8028']

In [21]:
torch.cuda.device(0)

In [32]:
lr = torch.from_numpy(np.asarray(list(range(10))))

In [34]:
torch.index_select(tr, 1, lr)

tensor([[116, 113, 113,  ..., 116, 113, 116],
        [112, 114, 110,  ..., 116, 113, 114],
        [110, 113, 112,  ..., 113, 110, 115],
        ...,
        [109, 114, 113,  ..., 110, 111, 114],
        [113, 111, 113,  ..., 110, 112, 114],
        [111, 112, 112,  ..., 112, 109, 111]], dtype=torch.int16)

In [60]:
from numba import njit

In [79]:
def get_mean_intensity(r):
    result_ar[r] = (tr[lab == r+1]).numpy().mean()
    return result_ar

In [72]:
%%time
v = get_mean_intensity()

CPU times: user 1min 3s, sys: 4.98 s, total: 1min 8s
Wall time: 3.42 s


In [88]:
torch.index_select(ar, 0, lr == 1)

TypeError: index_select() received an invalid combination of arguments - got (numpy.ndarray, int, list), but expected one of:
 * (Tensor input, int dim, Tensor index, *, Tensor out)
 * (Tensor input, name dim, Tensor index, *, Tensor out)


In [89]:
ar

array([[116, 113, 113, ..., 121, 115, 117],
       [112, 114, 110, ..., 116, 115, 118],
       [110, 113, 112, ..., 118, 115, 116],
       ...,
       [109, 114, 113, ..., 111, 113, 111],
       [113, 111, 113, ..., 112, 115, 111],
       [111, 112, 112, ..., 115, 116, 111]], dtype=int16)